# Market Risk Project – ESILV – 2025-2026

JONDEAU Martin - BAYRE Adrien

### Question A 

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm, gaussian_kde

In [3]:
df = pd.read_csv("Natixis Stock.csv", header=None, sep=r"\s+", names=["Date", "Price"])
df.head()

,Date,Price
0,02/01/2015,"5,621"
1,05/01/2015,"5,424"
2,06/01/2015,"5,329"
3,07/01/2015,"5,224"
4,08/01/2015,"5,453"


In [4]:
df["Date"]=pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce")
df["Price"]=df["Price"].str.replace(",",".").astype(float)

In [7]:
print(df.head(5))
print(df.tail(5))
print(df.describe())
print("Number of NaN price:", df["Price"].isna().sum())

        Date  Price
0 2015-01-02  5.621
1 2015-01-05  5.424
2 2015-01-06  5.329
3 2015-01-07  5.224
4 2015-01-08  5.453
           Date  Price
1018 2018-12-21  4.045
1019 2018-12-24  4.010
1020 2018-12-27  3.938
1021 2018-12-28  4.088
1022 2018-12-31  4.119
                                Date        Price
count                           1023  1023.000000
mean   2016-12-30 10:54:32.727272704     5.684600
min              2015-01-02 00:00:00     3.077000
25%              2016-01-02 00:00:00     4.927000
50%              2016-12-29 00:00:00     5.782000
75%              2017-12-28 12:00:00     6.532000
max              2018-12-31 00:00:00     7.744000
std                              NaN     1.021034
Number of NaN price: 0


In [8]:
# la date va que jusqu'à decembre 2016
df =df[(df["Date"]>="2015-01-01") & (df["Date"]<="2016-12-31")]
df = df.iloc[1:,:]
print(df.tail(5))

          Date  Price
508 2016-12-23  5.376
509 2016-12-27  5.380
510 2016-12-28  5.379
511 2016-12-29  5.328
512 2016-12-30  5.360


In [11]:
alpha=0.05

In [ ]:
df["return"]=df["Price"].pct_change()
df = df.iloc[1:] #del the Nan values of the first row
df

C:\Users\Msi Gaming\AppData\Local\Temp\ipykernel_40428\3890399504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["return"]=df["Price"].pct_change()


,Date,Price,return
3,2015-01-07,5.224,-0.019704
4,2015-01-08,5.453,0.043836
5,2015-01-09,5.340,-0.020723
6,2015-01-12,5.264,-0.014232
7,2015-01-13,5.400,0.025836
...,...,...,...
508,2016-12-23,5.376,-0.008118
509,2016-12-27,5.380,0.000744
510,2016-12-28,5.379,-0.000186
511,2016-12-29,5.328,-0.009481


In [ ]:
import numpy as np
def K(x):

    return (15/16)*(1-x*x)

def F(x,rend):
    n= len(rend)
    h= 1.06*np.std(rend)*(n**(-1/5))
    somme=0
    for i in range(len(rend)):
        u=(x-rend[i])/h

        if (abs(u)<=1) :
           somme+=K(u)
        else:
            somme+=0

    return somme/len(rend)


def dichotomie(rend,alpha):
    a=-1
    b=1
    eps = 0.000000001
    while abs(b - a) > eps:
        m = (a + b)/2
        if F(m,rend) == 1-alpha:
            return m
        elif F(m,rend) < 1-alpha:
            a = m
        else:
            b = m
    return (a + b) / 2

Var_Nonpara=dichotomie(df['Price Returns'], 0.95)
print('VaR Non-parametric at 95%  =',Var_Nonpara)

#print('1-alpha =',F(Var_Nonpara,df['Price Returns']))
#On travaille sur des pertes donc on va prendre -VaR pour avoir une VaR positive
Var_Nonpara=-Var_Nonpara


Empirical VaR at 95.00% : -3.788407989779058%


In [24]:
#var paramétrique --> peut avoir des erreurs de paramètres
#on utilise donc la var non param pour eviter ces erreurs de paramètre 